In [2]:
import requests
import pandas as pd
df=pd.read_csv('data.csv')
df=pd.DataFrame(df)

In [8]:
from geopy.geocoders import Nominatim
import time
locator=Nominatim(user_agent="myGeocoder")
lat=[]
lon=[]
def findloc(address):
  location=locator.geocode(address)
  time.sleep(1)
  if(location):
    lat.append(location.latitude)
    lon.append(location.longitude)

for i in range(0,77):
  row=df.iloc[i]
  findloc(row["address"])
coordinates = list(zip(lat, lon))

unique_coordinates = list(set(coordinates))

unique_lat, unique_long = zip(*unique_coordinates)

unique_lat = list(unique_lat)
unique_long = list(unique_long)


In [12]:
import random
def generate_clustered_coordinates(hospital, variance=0.01):
    lat, lon = hospital
    lat += random.uniform(-variance, variance)
    lon += random.uniform(-variance, variance)
    return round(lat, 6), round(lon, 6)
data = []
for i in range(75):
    numberplate = f"KA-{random.randint(1, 99):02d}-{random.choice(['A', 'B', 'C', 'D'])}{random.randint(1000, 9999)}"
    hospital = random.choice(list(zip(unique_lat,unique_long)))  # Randomly choose a hospital to cluster around
    lat, lon = generate_clustered_coordinates(hospital, variance=0.01)  # Add noise
    data.append({"Ambulance_Numberplate": numberplate, "Latitude": lat, "Longitude": lon})

# Convert to DataFrame
df = pd.DataFrame(data)
df

def generate_type():
  ambulance_types = ['ALS', 'BLS', 'Non-Emergency']
  probabilities = [0.30, 0.50, 0.20]
  df['ambulance_type'] = np.random.choice(ambulance_types, size=len(df), p=probabilities)

import numpy as np
generate_type()

df['Ventilator']=df['ambulance_type'].apply(lambda x: 1 if ((x=='ALS')) else 0)
df['Oxygen']=df['ambulance_type'].apply(lambda x: 1 if ((x=='ALS') | (x=='BLS')) else 0)
df['Cardiac']=df['ambulance_type'].apply(lambda x: 1 if ((x=='ALS') | (x=='BLS')) else 0)
df
import folium
m=folium.Map(location=[12.9453,77.54533],zoom_start=10)
for i,row in df.iterrows():
  lats=df.at[i,'Latitude']
  lons=df.at[i,'Longitude']
  amb=df.at[i,'Ambulance_Numberplate']

  folium.Marker(location=[lats,lons],popup=amb,icon=folium.Icon(color="green")).add_to(m)
folium.Marker(location=[12.9453,77.54533],popup="My Location",icon=folium.Icon(color="red")).add_to(m)
m

In [13]:
df=df.to_csv('ambulance.csv')